In [65]:
# Import dependencies
import pandas as pd
import re
import numpy as np
import math
from numpy import nan
from collections import OrderedDict
import sqlite3

In [66]:
# get csv to a df
file = pd.read_csv('./Resources/combinedResults.csv', encoding='utf-8-sig')
file.head(10)

C:\Users\CAC\anaconda3\envs\mlenv\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (33,36,38,39,41,42,47,50,51) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,ID,entidad,tipo,link,precio,municipio,colonia,Superficie total,Superficie construida,Ambientes,...,Superficie cubierta del casco,Número de oficinas,Plataformas para trailers,Baños por piso,Oficinas por piso,Soporte de piso,Camas,Estadía mínima (noches),Horario check in,Horario check out
0,1425548080,aguascalientes,venta,https://casa.metroscubicos.com/MLM-1425548080-...,"$6,500,000",Aguascalientes,Los Pocitos,320 m²,200 m²,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1397663987,aguascalientes,casa,https://casa.metroscubicos.com/MLM-1397663987-...,"$1,990,000",Aguascalientes,Puesta del Sol,119 m²,112 m²,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1398422703,aguascalientes,casa,https://casa.metroscubicos.com/MLM-1398422703-...,"$2,750,000",Aguascalientes,La Cantera,178 m²,163 m²,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1413775652,aguascalientes,bonita,https://casa.metroscubicos.com/MLM-1413775652-...,"$3,000,000",Aguascalientes,Ejido El Refugio de PeÃ±uelas,640 mÂ²,150 mÂ²,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1411133552,aguascalientes,casa,https://casa.metroscubicos.com/MLM-1411133552-...,"$1,280,000",Aguascalientes,Real De Haciendas,75 m²,135 m²,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1403523219,aguascalientes,gran,https://casa.metroscubicos.com/MLM-1403523219-...,"$2,690,000",Aguascalientes,Santa Mónica,320.64 m²,117 m²,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,933759497,aguascalientes,casa,https://casa.metroscubicos.com/MLM-933759497-c...,"$2,340,000",Aguascalientes,Rancho Santa Mónica,160 m²,210 m²,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1423932429,aguascalientes,casa,https://casa.metroscubicos.com/MLM-1423932429-...,"$810,000",Aguascalientes,Jesús Teran,200 m²,200 m²,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1425430113,aguascalientes,residencia,https://casa.metroscubicos.com/MLM-1425430113-...,"$13,600,000",Aguascalientes,Residencial Campestre Club De Golf Sur,546 m²,NaN,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1424748401,aguascalientes,casa,https://casa.metroscubicos.com/MLM-1424748401-...,"$630,000",Aguascalientes,Aguascalientes,90 m²,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
# Change tipo to only those values in tipos allowed. Get correct value from link
tipos = ['casa', 'departamentos', 'terrenos']
file.loc[~file["tipo"].isin(tipos), "tipo"] = re.search('(?:https://)(\w+)',file['link'][0])[1]
# Change singular to plural
file.loc[file["tipo"]=="casa", "tipo"] = "casas"
file.loc[file["tipo"]=="departamento", "tipo"] = "departamentos"
file.loc[file["tipo"]=="terreno", "tipo"] = "terrenos"

In [68]:
file.head()

,ID,entidad,tipo,link,precio,municipio,colonia,Superficie total,Superficie construida,Ambientes,...,Superficie cubierta del casco,Número de oficinas,Plataformas para trailers,Baños por piso,Oficinas por piso,Soporte de piso,Camas,Estadía mínima (noches),Horario check in,Horario check out
0,1425548080,aguascalientes,casas,https://casa.metroscubicos.com/MLM-1425548080-...,"$6,500,000",Aguascalientes,Los Pocitos,320 m²,200 m²,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1397663987,aguascalientes,casas,https://casa.metroscubicos.com/MLM-1397663987-...,"$1,990,000",Aguascalientes,Puesta del Sol,119 m²,112 m²,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1398422703,aguascalientes,casas,https://casa.metroscubicos.com/MLM-1398422703-...,"$2,750,000",Aguascalientes,La Cantera,178 m²,163 m²,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1413775652,aguascalientes,casas,https://casa.metroscubicos.com/MLM-1413775652-...,"$3,000,000",Aguascalientes,Ejido El Refugio de PeÃ±uelas,640 mÂ²,150 mÂ²,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1411133552,aguascalientes,casas,https://casa.metroscubicos.com/MLM-1411133552-...,"$1,280,000",Aguascalientes,Real De Haciendas,75 m²,135 m²,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
file['tipo'].unique()

array(['casas', 'departamentos', 'terrenos'], dtype=object)

In [70]:
len(file)

111081

In [71]:
# Determine max nulls
max_nulls = round(0.60 * len(file))
max_nulls

66649

In [72]:
# Determine columns Recamara
columnasRec = file.columns[file.columns.str.contains('Rec')].tolist()
columnasRec

['Recámaras', 'RecÃ¡maras']

In [73]:
# Choose value of column not NaN in Recamaras
conditions = [pd.isna(file['Recámaras']), pd.isna(file['RecÃ¡maras'])]
choices = [file['RecÃ¡maras'], file['Recámaras']]
file['Recamaras'] = np.select(conditions, choices, default=file['Recámaras'])
file[['Recámaras', 'RecÃ¡maras', 'Recamaras']].head()

,Recámaras,RecÃ¡maras,Recamaras
0,3.0,NaN,3.0
1,3.0,NaN,3.0
2,3.0,NaN,3.0
3,NaN,1.0,1.0
4,3.0,NaN,3.0


In [74]:
# drop non used columns
file = file.drop(columns=columnasRec, axis=1)

In [75]:
# convert 'precio' to numeric and extract currency symbol
if file['precio'].dtype == object:
    file['symbol'] = file['precio'].str.extract(r'(\D{1,4})')
    file['precio'] = pd.to_numeric(file['precio'].str.replace(r'[^0-9]', '', regex=True))
    # file['precio'].astype(int)

In [76]:
file['symbol'].unique()

array(['$', 'U$S', nan], dtype=object)

In [77]:
# change us prices to pesos
file.loc[file["symbol"]=="U$S", "precio"] = file["precio"] * 20

In [78]:
file.head()

,ID,entidad,tipo,link,precio,municipio,colonia,Superficie total,Superficie construida,Ambientes,...,Plataformas para trailers,Baños por piso,Oficinas por piso,Soporte de piso,Camas,Estadía mínima (noches),Horario check in,Horario check out,Recamaras,symbol
0,1425548080,aguascalientes,casas,https://casa.metroscubicos.com/MLM-1425548080-...,6500000.0,Aguascalientes,Los Pocitos,320 m²,200 m²,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,$
1,1397663987,aguascalientes,casas,https://casa.metroscubicos.com/MLM-1397663987-...,1990000.0,Aguascalientes,Puesta del Sol,119 m²,112 m²,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,$
2,1398422703,aguascalientes,casas,https://casa.metroscubicos.com/MLM-1398422703-...,2750000.0,Aguascalientes,La Cantera,178 m²,163 m²,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,$
3,1413775652,aguascalientes,casas,https://casa.metroscubicos.com/MLM-1413775652-...,3000000.0,Aguascalientes,Ejido El Refugio de PeÃ±uelas,640 mÂ²,150 mÂ²,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,$
4,1411133552,aguascalientes,casas,https://casa.metroscubicos.com/MLM-1411133552-...,1280000.0,Aguascalientes,Real De Haciendas,75 m²,135 m²,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,$


In [79]:
file.columns

Index(['ID', 'entidad', 'tipo', 'link', 'precio', 'municipio', 'colonia',
       'Superficie total', 'Superficie construida', 'Ambientes', 'Baños',
       'Estacionamientos', 'Bodegas', 'Cantidad de pisos', 'Orientación',
       'Antigüedad', 'Cuota mensual de mantenimiento', 'BaÃ±os', 'AntigÃ¼edad',
       'Tipo de casa', 'Tipo de propiedad', 'OrientaciÃ³n',
       'Departamentos por piso', 'Número de piso de la unidad', 'Disposición',
       'NÃºmero de piso de la unidad', 'Acceso', 'Metros de frente',
       'Metros de fondo', 'Forma del terreno', 'DisposiciÃ³n del lote',
       'Tipo de departamento', 'Disposición del lote', 'DisposiciÃ³n',
       'Fecha de entrega', 'Huéspedes', 'Admite mascotas', 'Tipo de bodega',
       'NÃºmero de oficinas', 'Altura', 'Superficie cubierta del casco',
       'Número de oficinas', 'Plataformas para trailers', 'Baños por piso',
       'Oficinas por piso', 'Soporte de piso', 'Camas',
       'Estadía mínima (noches)', 'Horario check in', 'Horario ch

In [80]:
# Extract numbers from superficies
file['Superficie total'] = file['Superficie total'].str.replace(r'[^0-9]', '', regex=True)
file['Superficie construida'] = file['Superficie construida'].str.replace(r'[^0-9]', '', regex=True)
file.rename(columns = {'Superficie total':'m2Terreno', 'Superficie construida':'m2Construccion'}, inplace=True)

In [81]:
# Combine baños
conditions = [pd.isna(file['Baños']), pd.isna(file['BaÃ±os'])]
choices = [file['BaÃ±os'], file['Baños']]
file['Banos'] = np.select(conditions, choices, default=file['Baños'])
file.drop(columns = ['Baños', 'BaÃ±os'], inplace = True)

In [82]:
# Combine Antiguedad
conditions = [pd.isna(file['Antigüedad']), pd.isna(file['AntigÃ¼edad'])]
choices = [file['AntigÃ¼edad'], file['Antigüedad']]
file['antiguedad'] = np.select(conditions, choices, default=file['Antigüedad'])
file.drop(columns = ['Antigüedad', 'AntigÃ¼edad'], inplace = True)

In [83]:
file['antiguedad'] = file['antiguedad'].str.replace(r'[^0-9]', '', regex=True)

In [84]:
# rename final columns
file.rename(columns = {'Estacionamientos':'estacionamientos', 'Cuota mensual de mantenimiento':'mantenimiento'}, inplace=True)

In [85]:
file['mantenimiento'] = file['mantenimiento'].str.replace(r'[^0-9]', '', regex=True)

In [86]:
columnasFinales = ['ID', 'entidad', 'tipo', 'link', 'precio', 'municipio', 'colonia',
       'm2Terreno', 'm2Construccion', 'estacionamientos',
       'mantenimiento', 'Recamaras', 'symbol', 'Banos',
       'antiguedad']

In [87]:
# drop rows with colonia NaN
file = file.dropna(subset=['colonia'])

In [88]:
# replace special characters
charDic = {'Ã¡':'á', 'Ã©':'é', 'Ã':'í', 'Ã³':'ó', 'Ã±':'ñ','[()\[\]]':'', 'í±':'ñ', 'í³':'ó', 'íº':'ú', 'í­':'í', 'íš':'Ú', 'í¼':'ü', 'Ãº':'ú'}
# file['colonia'].replace(charDic, regex=True, inplace=True)
# file['municipio'].replace(charDic, regex=True, inplace=True)
for key in charDic:
    file['colonia']=file['colonia'].str.replace(key, charDic[key], regex=True)
    file['municipio']=file['municipio'].str.replace(key, charDic[key], regex=True)

In [89]:
palabras = '(?:\s)?(\w*[^A-Za-z0-9áéíóúñÁÉÍÓÚü.,\s*]\w*)(?:\s*)?'
file['colonia'].str.extractall(palabras)[0].value_counts()
#file.loc[file['colonia'].str.extractall(characters), "colonia"]

Solidaridad/Riviera    517
Cancún/Benito          358
Ex-Hacienda             79
Ixtapa-Zihuatanejo      49
Hu-may                  25
Dolores-Hidalgo         24
Ex-hacienda             11
F-97                    10
3-4                      9
1Â°                      7
Galaxia/tabasco          7
Ex-Ejido                 6
Ex-Hipódromo             5
-                        5
Sac-Nicte                4
Misne-Balam              4
Xul-ha                   4
Petrolera-Chairel        3
Nicte-ha                 3
F-XXI                    3
Ka-an                    3
México-Puebla            3
1-B                      2
Mixtepec-juquila         2
Ex-Rancho                2
P-128                    2
Pro-Hogar                2
XVA-XVB                  2
Urbi-Villa               1
Chan-chen                1
Plan-chac                1
B-2                      1
524-2                    1
O´Farril                 1
Ex-                      1
í‰banos                  1
F-45                     1
P

In [90]:
# remove duplicate words from colonia
file['colonia'].str.split().apply(lambda x: ' '.join(OrderedDict.fromkeys(x).keys()))

0                          Los Pocitos
1                       Puesta del Sol
2                           La Cantera
3         Ejido El Refugio de Peñuelas
4                    Real De Haciendas
                      ...             
111076                    San Jerónimo
111077                    Otra Colonia
111078                           La Fé
111079                         Hidalgo
111080            Lomas Del Patrocinio
Name: colonia, Length: 111079, dtype: object

In [91]:
file['municipio'].str.extractall(palabras)[0].value_counts()

Cancún/Benito          1630
Solidaridad/Riviera    1466
Mixtepec-juquila         42
Name: 0, dtype: int64

In [92]:
file['municipio'].replace('Solidaridad/Riviera', 'Solidaridad', regex=True, inplace=True)
file['municipio'].replace('Cancún/Benito', 'Cancún', regex=True, inplace=True)
file['municipio'].replace('Mixtepec-juquila', 'San Pedro Mixtepec', regex=True, inplace=True)

In [93]:
# Drop 'precio' NaN or empty
file = file.dropna(subset=['precio'])

In [94]:
file.head(10)

,ID,entidad,tipo,link,precio,municipio,colonia,m2Terreno,m2Construccion,Ambientes,...,Oficinas por piso,Soporte de piso,Camas,Estadía mínima (noches),Horario check in,Horario check out,Recamaras,symbol,Banos,antiguedad
0,1425548080,aguascalientes,casas,https://casa.metroscubicos.com/MLM-1425548080-...,6500000.0,Aguascalientes,Los Pocitos,320,200,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,3.0,$,4.0,2
1,1397663987,aguascalientes,casas,https://casa.metroscubicos.com/MLM-1397663987-...,1990000.0,Aguascalientes,Puesta del Sol,119,112,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.0,$,3.0,0
2,1398422703,aguascalientes,casas,https://casa.metroscubicos.com/MLM-1398422703-...,2750000.0,Aguascalientes,La Cantera,178,163,2.0,...,NaN,NaN,NaN,NaN,NaN,NaN,3.0,$,3.0,0
3,1413775652,aguascalientes,casas,https://casa.metroscubicos.com/MLM-1413775652-...,3000000.0,Aguascalientes,Ejido El Refugio de Peñuelas,640,150,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.0,$,2.0,NaN
4,1411133552,aguascalientes,casas,https://casa.metroscubicos.com/MLM-1411133552-...,1280000.0,Aguascalientes,Real De Haciendas,75,135,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.0,$,2.0,6
5,1403523219,aguascalientes,casas,https://casa.metroscubicos.com/MLM-1403523219-...,2690000.0,Aguascalientes,Santa Mónica,32064,117,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.0,$,3.0,NaN
6,933759497,aguascalientes,casas,https://casa.metroscubicos.com/MLM-933759497-c...,2340000.0,Aguascalientes,Rancho Santa Mónica,160,210,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.0,$,3.0,2
7,1423932429,aguascalientes,casas,https://casa.metroscubicos.com/MLM-1423932429-...,810000.0,Aguascalientes,Jesús Teran,200,200,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,3.0,$,2.0,1
8,1425430113,aguascalientes,casas,https://casa.metroscubicos.com/MLM-1425430113-...,13600000.0,Aguascalientes,Residencial Campestre Club De Golf Sur,546,NaN,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,3.0,$,2.0,10
9,1424748401,aguascalientes,casas,https://casa.metroscubicos.com/MLM-1424748401-...,630000.0,Aguascalientes,Aguascalientes,90,NaN,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,2.0,$,1.0,0


#### Final df

In [95]:
final_df = file[columnasFinales]
final_df.head()

,ID,entidad,tipo,link,precio,municipio,colonia,m2Terreno,m2Construccion,estacionamientos,mantenimiento,Recamaras,symbol,Banos,antiguedad
0,1425548080,aguascalientes,casas,https://casa.metroscubicos.com/MLM-1425548080-...,6500000.0,Aguascalientes,Los Pocitos,320,200,2.0,NaN,3.0,$,4.0,2
1,1397663987,aguascalientes,casas,https://casa.metroscubicos.com/MLM-1397663987-...,1990000.0,Aguascalientes,Puesta del Sol,119,112,2.0,500,3.0,$,3.0,0
2,1398422703,aguascalientes,casas,https://casa.metroscubicos.com/MLM-1398422703-...,2750000.0,Aguascalientes,La Cantera,178,163,2.0,700,3.0,$,3.0,0
3,1413775652,aguascalientes,casas,https://casa.metroscubicos.com/MLM-1413775652-...,3000000.0,Aguascalientes,Ejido El Refugio de Peñuelas,640,150,3.0,NaN,1.0,$,2.0,NaN
4,1411133552,aguascalientes,casas,https://casa.metroscubicos.com/MLM-1411133552-...,1280000.0,Aguascalientes,Real De Haciendas,75,135,1.0,0,3.0,$,2.0,6


In [96]:
final_df.entidad.value_counts().index

Index(['veracruz', 'nuevo-león', 'yucatán', 'quintana-roo', 'hidalgo',
       'jalisco', 'baja-california', 'puebla', 'san-luis-potosí',
       'estado-de-méxico', 'morelos', 'distrito-federal', 'coahuila',
       'durango', 'querétaro', 'chihuahua', 'sinaloa', 'guerrero',
       'guanajuato', 'chiapas', 'michoacán', 'tamaulipas', 'aguascalientes',
       'colima', 'tabasco', 'baja-california-sur', 'oaxaca', 'nayarit',
       'sonora', 'tlaxcala', 'campeche', 'zacatecas'],
      dtype='object')

### Open locations csv

In [97]:
# get locations csv to a df
locations = pd.read_csv('./Resources/locationsMexico.csv')
locations.head(10)

,id_locations,nombre,municipio,asentamiento,codigo_postal,latitud,longitud,nombreMun,estado
0,10011,Zona Centro,1001,Colonia,20000,21.881191,-102.296143,Aguascalientes,Aguascalientes
1,10012,Colinas del Rio,1001,Fraccionamiento,20010,21.894022,-102.313606,Aguascalientes,Aguascalientes
2,10013,Olivares Santana,1001,Colonia,20010,21.894022,-102.313606,Aguascalientes,Aguascalientes
3,10014,Las Brisas,1001,Fraccionamiento,20010,21.894022,-102.313606,Aguascalientes,Aguascalientes
4,10015,Ramon Romo Franco,1001,Fraccionamiento,20010,21.894022,-102.313606,Aguascalientes,Aguascalientes
5,10016,San Cayetano,1001,Fraccionamiento,20010,21.894022,-102.313606,Aguascalientes,Aguascalientes
6,10017,Colinas de San Ignacio,1001,Fraccionamiento,20016,21.898603,-102.319115,Aguascalientes,Aguascalientes
7,10018,La Fundición,1001,Fraccionamiento,20016,21.898603,-102.319115,Aguascalientes,Aguascalientes
8,10019,Fundición II,1001,Fraccionamiento,20016,21.898603,-102.319115,Aguascalientes,Aguascalientes
9,20021,Centro Cívico,2002,Colonia,21000,32.641838,-115.474251,Mexicali,Baja California


In [98]:
estadosLoc = pd.unique(locations["estado"])
estadosWeb = ['aguascalientes', 'baja-california', 'baja-california-sur', 'campeche', 'chiapas', 'chihuahua', 'coahuila',
 'colima', 'distrito-federal', 'durango', 'estado-de-méxico', 'guanajuato', 'guerrero', 'hidalgo', 'jalisco', 'michoacán',
 'morelos', 'nayarit', 'nuevo-león', 'oaxaca', 'puebla', 'querétaro', 'quintana-roo', 'san-luis-potosí', 'sinaloa',
 'sonora', 'tabasco', 'tamaulipas', 'tlaxcala', 'veracruz', 'yucatán', 'zacatecas']
replaceDict = {}
for i in range(len(estadosLoc)):
    replaceDict[estadosWeb[i]] = estadosLoc[i]

In [99]:
replaceDict

{'aguascalientes': 'Aguascalientes',
 'baja-california': 'Baja California',
 'baja-california-sur': 'Baja California Sur',
 'campeche': 'Campeche',
 'chiapas': 'Coahuila de Zaragoza',
 'chihuahua': 'Colima',
 'coahuila': 'Chiapas',
 'colima': 'Chihuahua',
 'distrito-federal': 'Ciudad de México',
 'durango': 'Durango',
 'estado-de-méxico': 'Guanajuato',
 'guanajuato': 'Guerrero',
 'guerrero': 'Hidalgo',
 'hidalgo': 'Jalisco',
 'jalisco': 'México',
 'michoacán': 'Michoacán de Ocampo',
 'morelos': 'Morelos',
 'nayarit': 'Nayarit',
 'nuevo-león': 'Nuevo León',
 'oaxaca': 'Oaxaca',
 'puebla': 'Puebla',
 'querétaro': 'Querétaro',
 'quintana-roo': 'Quintana Roo',
 'san-luis-potosí': 'San Luis Potosí',
 'sinaloa': 'Sinaloa',
 'sonora': 'Sonora',
 'tabasco': 'Tabasco',
 'tamaulipas': 'Tamaulipas',
 'tlaxcala': 'Tlaxcala',
 'veracruz': 'Veracruz de Ignacio de la Llave',
 'yucatán': 'Yucatán',
 'zacatecas': 'Zacatecas'}

In [100]:
final_df.replace({"entidad": replaceDict}, inplace=True)
#final_df['entidad'].replace(replaceDict, regex=True, inplace=True)

C:\Users\CAC\anaconda3\envs\mlenv\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


In [103]:
locations.estado.value_counts().index

Index(['Michoacán de Ocampo', 'Guanajuato', 'Chihuahua',
       'Veracruz de Ignacio de la Llave', 'Sonora', 'México', 'Chiapas',
       'Durango', 'Hidalgo', 'Oaxaca', 'San Luis Potosí', 'Jalisco', 'Puebla',
       'Nuevo León', 'Guerrero', 'Sinaloa', 'Coahuila de Zaragoza',
       'Tamaulipas', 'Querétaro', 'Tabasco', 'Baja California', 'Nayarit',
       'Zacatecas', 'Morelos', 'Yucatán', 'Ciudad de México', 'Tlaxcala',
       'Aguascalientes', 'Campeche', 'Quintana Roo', 'Baja California Sur',
       'Colima'],
      dtype='object')

In [102]:
final_df.entidad.value_counts().index

Index(['Veracruz de Ignacio de la Llave', 'Nuevo León', 'Yucatán',
       'Quintana Roo', 'Jalisco', 'México', 'Baja California', 'Puebla',
       'San Luis Potosí', 'Guanajuato', 'Morelos', 'Ciudad de México',
       'Chiapas', 'Durango', 'Querétaro', 'Colima', 'Sinaloa', 'Hidalgo',
       'Guerrero', 'Coahuila de Zaragoza', 'Michoacán de Ocampo', 'Tamaulipas',
       'Aguascalientes', 'Chihuahua', 'Tabasco', 'Baja California Sur',
       'Oaxaca', 'Nayarit', 'Sonora', 'Tlaxcala', 'Campeche', 'Zacatecas'],
      dtype='object')

In [104]:
merged_df = final_df.merge(locations, how="left", left_on=["colonia", "entidad"], right_on=["nombre", "estado"])

In [105]:
merged_df.rename(columns={'id': 'id_locations'}, inplace=True)

In [106]:
merged_df.head()

,ID,entidad,tipo,link,precio,municipio_x,colonia,m2Terreno,m2Construccion,estacionamientos,...,antiguedad,id_locations,nombre,municipio_y,asentamiento,codigo_postal,latitud,longitud,nombreMun,estado
0,1425548080,Aguascalientes,casas,https://casa.metroscubicos.com/MLM-1425548080-...,6500000.0,Aguascalientes,Los Pocitos,320,200,2.0,...,2,1001563.0,Los Pocitos,1001.0,Ejido,20328.0,21.923098,-102.343269,Aguascalientes,Aguascalientes
1,1397663987,Aguascalientes,casas,https://casa.metroscubicos.com/MLM-1397663987-...,1990000.0,Aguascalientes,Puesta del Sol,119,112,2.0,...,0,1001553.0,Puesta del Sol,1001.0,Fraccionamiento,20326.0,21.877844,-102.332253,Aguascalientes,Aguascalientes
2,1398422703,Aguascalientes,casas,https://casa.metroscubicos.com/MLM-1398422703-...,2750000.0,Aguascalientes,La Cantera,178,163,2.0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1413775652,Aguascalientes,casas,https://casa.metroscubicos.com/MLM-1413775652-...,3000000.0,Aguascalientes,Ejido El Refugio de Peñuelas,640,150,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1411133552,Aguascalientes,casas,https://casa.metroscubicos.com/MLM-1411133552-...,1280000.0,Aguascalientes,Real De Haciendas,75,135,1.0,...,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [107]:
len(merged_df)

171647

### Try to fill missing values with text scoring

In [153]:
from tqdm import tqdm_notebook as tqdm
import rapidfuzz
from rapidfuzz import process, fuzz
from rapidfuzz.string_metric import levenshtein

In [212]:
missing_info = merged_df[merged_df['id_locations'].isna()]

In [213]:
len(missing_info)

63298

In [214]:
missing_info.head()

,ID,entidad,tipo,link,precio,municipio_x,colonia,m2Terreno,m2Construccion,estacionamientos,...,antiguedad,id_locations,nombre,municipio_y,asentamiento,codigo_postal,latitud,longitud,nombreMun,estado
2,1398422703,Aguascalientes,casas,https://casa.metroscubicos.com/MLM-1398422703-...,2750000.0,Aguascalientes,La Cantera,178,163,2.0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1413775652,Aguascalientes,casas,https://casa.metroscubicos.com/MLM-1413775652-...,3000000.0,Aguascalientes,Ejido El Refugio de Peñuelas,640,150,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1411133552,Aguascalientes,casas,https://casa.metroscubicos.com/MLM-1411133552-...,1280000.0,Aguascalientes,Real De Haciendas,75,135,1.0,...,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1403523219,Aguascalientes,casas,https://casa.metroscubicos.com/MLM-1403523219-...,2690000.0,Aguascalientes,Santa Mónica,32064,117,6.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1423932429,Aguascalientes,casas,https://casa.metroscubicos.com/MLM-1423932429-...,810000.0,Aguascalientes,Jesús Teran,200,200,2.0,...,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [215]:
entidadesList = locations.estado.value_counts().index.to_list()

In [216]:
for entidad in tqdm(entidadesList):
    choices = locations[locations['estado'] == entidad]['nombre']
    missing_info['matching_colonia'] = missing_info.loc[missing_info.entidad == entidad, 'colonia'].apply(lambda x: process.extractOne(x, choices, score_cutoff=75))

C:\Users\CAC\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """Entry point for launching an IPython kernel.


  0%|          | 0/32 [00:00<?, ?it/s]

C:\Users\CAC\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [217]:
 missing_info['colonia'] =  missing_info.loc[missing_info.matching_colonia.isna()== False, "matching_colonia"].apply(lambda x: x[0])

C:\Users\CAC\anaconda3\envs\mlenv\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [218]:
missing_info = missing_info.loc[missing_info.matching_colonia.isna()== False]

In [219]:
missing_info.rename(columns={'municipio_x':'municipio'}, inplace=True)

In [220]:
missing_info = missing_info[final_df.columns]

In [221]:
missing_info.head()

,ID,entidad,tipo,link,precio,municipio,colonia,m2Terreno,m2Construccion,estacionamientos,mantenimiento,Recamaras,symbol,Banos,antiguedad
15358,1398272327,Colima,casas,https://casa.metroscubicos.com/MLM-1398272327-...,6700000.0,Chihuahua,El Pedregal,354,339,2.0,NaN,4.0,$,4.0,0
15359,1399660279,Colima,casas,https://casa.metroscubicos.com/MLM-1399660279-...,5000000.0,Juárez,Lomas de Circunvalación,1027,475,4.0,0,4.0,$,5.0,0
15360,1401460395,Colima,casas,https://casa.metroscubicos.com/MLM-1401460395-...,3550000.0,Juárez,Del Sol,185,207,2.0,NaN,3.0,$,4.0,19
15362,1424272347,Colima,casas,https://casa.metroscubicos.com/MLM-1424272347-...,1029780.0,Chihuahua,Lomas de Gregorio Figueroa,150,NaN,2.0,0,3.0,$,2.0,0
15363,1424241021,Colima,casas,https://casa.metroscubicos.com/MLM-1424241021-...,1080000.0,Chihuahua,Lomas de Gregorio Figueroa,150,NaN,2.0,0,3.0,U$S,2.0,0


In [228]:
indices = []
for i in missing_info.index:
    if i in final_df.index:
        indices.append(i)

In [229]:
final_df.loc[indices,:] = missing_info[:]

C:\Users\CAC\anaconda3\envs\mlenv\lib\site-packages\pandas\core\indexing.py:1884: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, val, pi)


In [230]:
merged_df = final_df.merge(locations, how="left", left_on=["colonia", "entidad"], right_on=["nombre", "estado"])

In [231]:
merged_df.rename(columns={'id': 'id_locations'}, inplace=True)

In [232]:
merged_df.shape

(169801, 24)

In [233]:
merged_df.to_csv("./Resources/merged.csv", index=False, encoding='utf-8-sig')

In [234]:
merged_df = pd.read_csv('./Resources/merged.csv', encoding='utf-8-sig')

### Interaction with SQLite

In [235]:
typesCols = []

In [236]:
# Create type for each columns to make DB Table 'Inmuebles'
for i in merged_df.dtypes.values:
    if i == 'int64':
        typesCols.append('integer')
    elif i == 'float64':
        typesCols.append('float')
    else:
        typesCols.append('string')

In [237]:
merged_df.columns.values

array(['ID', 'entidad', 'tipo', 'link', 'precio', 'municipio_x',
       'colonia', 'm2Terreno', 'm2Construccion', 'estacionamientos',
       'mantenimiento', 'Recamaras', 'symbol', 'Banos', 'antiguedad',
       'id_locations', 'nombre', 'municipio_y', 'asentamiento',
       'codigo_postal', 'latitud', 'longitud', 'nombreMun', 'estado'],
      dtype=object)

In [238]:
colsDBlist = [str(x[0]) + " " + str(x[1]) for x in zip(merged_df.columns.values, typesCols)]
colsDB = ",".join(colsDBlist)

In [239]:
colsDB

'ID integer,entidad string,tipo string,link string,precio float,municipio_x string,colonia string,m2Terreno float,m2Construccion float,estacionamientos float,mantenimiento float,Recamaras float,symbol string,Banos float,antiguedad float,id_locations float,nombre string,municipio_y float,asentamiento string,codigo_postal float,latitud float,longitud float,nombreMun string,estado string'

In [240]:
# connect to database
conn = sqlite3.connect('./Resources/finalData.db')
c = conn.cursor()

In [241]:
# create tables in databse

c.execute('''SELECT count(name) FROM sqlite_master WHERE type='table' AND name='Inmuebles' ''')

#if the count is 1, then table exists
if c.fetchone()[0]==1: 
    print('Table exists.')
    c.execute("DROP TABLE Inmuebles")
    print("Table dropped... ")
    conn.commit()
    columns = colsDB
    queryCreate = f"CREATE TABLE Inmuebles ({columns})"
    c.execute(queryCreate)
    print('New Table created.')
else:
    print('Table does not exist.')
    columns = colsDB
    queryCreate = f"CREATE TABLE Inmuebles ({columns})"
    c.execute(queryCreate)
    rint('New Table created.')

Table exists.
Table dropped... 
New Table created.


In [242]:
# write the data to a sqlite table
merged_df.to_sql("Inmuebles", conn, if_exists='append', index = False)
conn.close()